In [14]:
from time import sleep
import cyschoolhousesuite as cysh
from pandas import read_excel

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

In [31]:
def search_for_section(driver, sec_num):
    driver.find_element_by_class_name("searchTextBox").clear()
    driver.find_element_by_class_name("searchTextBox").send_keys(sec_num)
    driver.find_element_by_name("search").click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "section_intervention_data")))
    driver.find_element_by_name("section_intervention_data").click()

def enrollment_window(driver):
    sleep(1)
    driver.switch_to_window(driver.window_handles[1])
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "addButton")))
    driver.find_element_by_id("addButton").click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "j_id0:j_id48:enrollDate")))
    driver.find_element_by_id("j_id0:j_id48:enrollDate").clear()
    driver.find_element_by_id("j_id0:j_id48:enrollDate").send_keys('8/14/2017')
    
def select_student_from_enrollment_table(driver, sfid):
    table = driver.find_element_by_xpath("//table[@id='availableStudents']/tbody")
    student = table.find_element_by_xpath("//tr[contains(@id, '" + sfid + "')]")
    student.click()
    sleep(2)
    
def enroll_batch(driver):
    driver.find_element_by_xpath("//img[@src='/resource/1496397506000/Style_Bundle/images/arrow-right.png']").click()

def update_gl(driver, _grade):
    selector = Select(driver.find_element_by_name("j_id0:j_id48:j_id103"))
    selector.select_by_visible_text(_grade)
    
def save_enrollments(driver):    
    driver.find_element_by_xpath("//input[@value='Save']").click()
    
def input_data():
    data = read_csv('input_files/section-key.csv')
    return data.to_dict(orient='records')

def test_script():
    inputs = input_data()[1]
    driver = cysh.get_driver()
    cysh.open_cyschoolhouse18_sb(driver)
    search_for_section(driver, 74013)
    enrollment_window(driver)
    select_student_from_enrollment_table(driver, inputs['Student ID'])
    enroll_batch(driver)
    #sleep(5)
    #driver.quit()

In [32]:
def enrollment_script():
    data = read_excel('input_files/student-enrollment-records.xlsx')
    staff = data['Intervention Primary Staff: Staff Name'].unique()
    for _staff in staff:
        driver = cysh.get_driver()
        cysh.open_cyschoolhouse18(driver)
        staff_records = data[data['Intervention Primary Staff: Staff Name'] == _staff]
        sections = staff_records['Intervention Section Number'].unique()
        for _section in sections:
            search_for_section(driver, str(_section))
            enrollment_window(driver)
            section_records = staff_records[staff_records['Intervention Section Number'] == _section]
            relevant_grades = section_records['Grade'].unique()
            for _grade in relevant_grades:
                update_gl(driver, str(_grade))
                sleep(2)
                students_in_gl = section_records[section_records.Grade == _grade]
                students_to_enroll = students_in_gl['Student ID'].unique()
                for _student in students_to_enroll:
                    select_student_from_enrollment_table(driver, _student)
                enroll_batch(driver)
            sleep(2)
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
        driver.quit()

In [33]:
enrollment_script()

TimeoutException: Message: 
